In [1]:
from sympy.solvers import solve
from sympy import symbols, summation, Sum, Function, init_printing
init_printing()

import scipy.optimize as spo

##### Future Value of Cash Today

The future value in T years of C dollars at the interest rate r compounded m times is equal to:

$$FV_{t} = C(1+\frac{r}{m})^{mT}$$

and the T-year compounding factor is:

$$CF_{t} = 1 + \frac{r}{m}^{mT}$$

In [10]:
def fv_of_cash_with_compound(cash, interest_rate, num_compounds, years):
    return round(cash*(1+(interest_rate/num_compounds))**(num_compounds*years), 2)

##### Example: 

What is the future value of present-day cash with an interest rate of 10%, compounding twice per year, in ten years?

In [11]:
fv_of_cash_with_compound(59436.96, 0.063, 1, 3)

71393.12

##### Effective Annual Interest Rate
Given the above, we can choose to think about investments that compound more often than annually and convert them into equivalent investments _as if_ they compounded annually. This is called the **effective annual rate** or the **annual percentage rate.**

##### Discounting and Present Value


In [7]:
def present_value_stream(discount_rate, cash_seq):
    # NOTE: Assumes that first payment is in period 1
    total_pv = 0
    for time_period, cash_flow in enumerate(cash_seq):
        total_pv += cash_flow/(1+discount_rate)**(time_period+1)
    return round(total_pv,2)

In [8]:
present_value_stream(.063, [11412,22824,34236])

59436.96

##### Perpetuities and Annuities

Present value of an annuity is:

$$PV_0 = \frac{C}{r} (1-\frac{1}{(1+r)^T})$$

In [2]:
def present_value_perpetuity(discount_rate, cash_flow, t_first_payment=1, growth_rate=0):
    """Present value of a perpetuity with optional arguments for a deferred first payment and a growth rate."""
    pv = cash_flow/(discount_rate-growth_rate)
    pv_discounted_back = pv*(1/(1+discount_rate)**(t_first_payment-1))
    return round(pv_discounted_back, 2)

In [14]:
present_value_perpetuity(.08, 400, 1)

5000.0

We can think of an annuity as being the difference between two perpetuities. 

In [16]:
def present_value_annuity(discount_rate, cash_flow, t_last_payment, t_first_payment=1, growth_rate=0):
    pv = cash_flow/discount_rate*(1-(1/(1+discount_rate)**t_last_payment))
    pv_discounted = pv*(1/(1+discount_rate)**(t_first_payment-1))
    return round(pv_discounted, 2)

In [40]:
def payment_per_period_annuity(target_pv, discount_rate, t_last_payment, t_first_payment=1, monthly_payment=1):
    """Work in progress, but this works recursively to identify what the right payment is for an annuity 
    rounded to the nearest integer (and from there you can work out what the right # of cents is)"""
    curr_pv1 = present_value_annuity(discount_rate, monthly_payment, t_last_payment, t_first_payment)
    curr_pv2 = present_value_annuity(discount_rate, monthly_payment+1, t_last_payment, t_first_payment)
    if target_pv - curr_pv1 < 1:
        return monthly_payment
    else:
        if abs(target_pv - curr_pv1) < abs(target_pv - curr_pv2):
            return payment_per_period_annuity(target_pv, discount_rate, t_last_payment, t_first_payment, monthly_payment-1)
        else:
            return payment_per_period_annuity(target_pv, discount_rate, t_last_payment, t_first_payment, monthly_payment+1)

In [41]:
payment_per_period_annuity(9000, .0047416, 60)

173

###### Problem Set 1 #3:

Suppose you deposit $1,000 in an account at the end of each of the next four years. If the account earns 12% annually, how much will be in the account at the end of seven years?

First, we should find the present value of the annuity. Then we can determine what the future value of that cash is in 7 years' time.

In [22]:
present_value_annuity(.12, 1000, 4)

In [18]:
fv_of_cash_with_compound(3037.35, .12, 1, 7)

Amerisave.com for mortgage rates

##### Investment Decision Rules

Net present value rule always leads to the investment decisions that are in the shareholder's best interest.

Discount rate is also known as the opportunity cost of capital, because it helps you understand whether your investment project can "beat" what it would do if you just invested that money at whatever that expected rate of return is for an alternative project.

In [10]:
def net_present_value(discount_rate, cash_seq):
    """The net present value of an investment is equivalent
    to the present value of the cash stream, being sure to include
    the cost of the investment within the stream itself
    """
    return present_value_stream(discount_rate, cash_seq)

**Example:** Consider a project with a 5,000 investment upfront and cash flows of 2,000 in time periods 1-3. What is the net present value of the project if the discount rate is 5%? Can you solve for the discount rate that would result in an NPV of 0?

In [11]:
net_present_value(.05, [-5000,2000,2000,2000])

##### Internal Rate of Return

##### Profitability Index Rule

The profitability index is defined as the NPV of future cash flows divided by the initial investment.

$$PI = \frac{NPV}{-C_{0}}$$

Note that the PI rule is equivalent to the NPV rule provided that C<sub>0</sub> < 0. We must flip the acceptance rule if we are thinking about borrowing projects.